In [4]:
%pip install aioarango
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [64]:
from aioarango import ArangoClient
import os
from dotenv import load_dotenv
from aioarango import errno, DocumentInsertError

load_dotenv()

username_arango = os.getenv('arango_username')
password_arango = os.getenv('arango_password')
database_arango = os.getenv('arango_database')
sys_username_arango = os.getenv('sys_username_arango')
sys_password_arango = os.getenv('sys_password_arango')

print(username_arango)
print(password_arango)
print(database_arango)

client = ArangoClient(hosts="https://arangodb.roulet.dev")

# Connect to "_system" database as root user.
sys_db = await client.db("_system", username=str(sys_username_arango), password=str(sys_password_arango))

# Create a new database named "DDI_MC2".
try:
    await sys_db.create_database("DDI_MC2")
except Exception as exc:
    if (exc.error_code == errno.DUPLICATE_NAME):
        print("Database already exists")


# Connect to "test" database as root user.
db = await client.db(str(database_arango), username=str(username_arango), password=str(password_arango))

# Create a new collection named "students".
try:
    users = await db.create_collection("users")
except Exception as e:
    if (e.error_code == errno.DUPLICATE_NAME):
        print("Collection already exists users")
        users = db.collection("users")

try:
    jwt_tokens = await db.create_collection("jwt_tokens")
except Exception as e:
    if (e.error_code == errno.DUPLICATE_NAME):
        print("Collection already exists jwt_tokens")
        jwt_tokens = db.collection("jwt_tokens")

try:
    books = await db.create_collection("books")
except Exception as e:
    if (e.error_code == errno.DUPLICATE_NAME):
        print("Collection already exists books")
        books = db.collection("movies")


user_model = [
    {
        "name": "John", "age": 25, "email": "sample@gmail.com", "adress": {"street": "street 1", "city": "city 1", "country": "Germany", "zip": "550232", "phone": "04423233232", "mobile": "04423233232", "fax": "04423233232", "website": "www.sample.com", "company": "sample company"}, "role": "admin", "status": "active", "created_at": "2021-05-01 12:00:00", "updated_at": "2021-05-01 12:00:00"
    },
]
book_model = [
    {
        "name": "Harry Potter", "author": "J.K. Rowling", "genre": "fantasy", "price": "10.00", "description": "The book is about"
    },
    {
      "name": "Lord of the Rings", "author": "J.R.R. Tolkien", "genre": "fantasy", "price": "10.00", "description": "The book is about"
    },
    { "name": "The Hobbit", "author": "J.R.R. Tolkien", "genre": "fantasy", "price": "10.00", "description": "The book is about" },
    { "name": "The Hitchhiker's Guide to the Galaxy", "author": "Douglas Adams", "genre": "fantasy", "price": "10.00", "description": "The book is about" },
    { "name": "The Lion, the Witch and the Wardrobe", "author": "C.S. Lewis", "genre": "fantasy", "price": "10.00", "description": "The book is about" },
    { "name": "The Da Vinci Code", "author": "Dan Brown", "genre": "fantasy", "price": "10.00", "description": "The book is about" },
    { "name": "The Alchemist", "author": "Paulo Coelho", "genre": "fantasy", "price": "10.00", "description": "The book is about" },
    { "name": "The Lord of the Rings", "author": "J.R.R. Tolkien", "genre": "fantasy", "price": "10.00", "description": "The book is about" },
    { "name": "The Little Prince", "author": "Antoine de Saint-Exupéry", "genre": "fantasy", "price": "10.00", "description": "The book is about" },
    { "name": "The Hobbit", "author": "J.R.R. Tolkien", "genre": "fantasy", "price": "10.00", "description": "The book is about" },
    { "name": "The Hitchhiker's Guide to the Galaxy", "author": "Douglas Adams", "genre": "fantasy", "price": "10.00", "description": "The book is about" },
]

for u in user_model:
    try:
        await users.insert(document=u, sync=True, return_new=True, silent=True, overwrite=True)
    except Exception as e:
        print(e)
        
for b in book_model:
    try:
        await books.insert(document=b, sync=True, return_new=True, silent=True, overwrite=True)
    except Exception as e:
        print(e)


DDI2
Welcome12
DDI_MC2
Database already exists
Collection already exists users
Collection already exists jwt_tokens
Collection already exists books
name 'books' is not defined
name 'books' is not defined
name 'books' is not defined
name 'books' is not defined
name 'books' is not defined
name 'books' is not defined
name 'books' is not defined
name 'books' is not defined
name 'books' is not defined
name 'books' is not defined
name 'books' is not defined


c:\Users\super\AppData\Local\Programs\Python\Python310\lib\site-packages\httpx\_client.py:1978: UserWarning: Unclosed <httpx.AsyncClient object at 0x000002035F1A7C70>. See https://www.python-httpx.org/async/#opening-and-closing-clients for details.
  warnings.warn(


In [32]:
cursor = await db.aql.execute("FOR u IN users RETURN u")
users = [document["name"] async for document in cursor]
print(users)

['jane', 'josh', 'judy', 'John', 'Jane', 'Jack', 'Jill', 'Jenny']
